In [2]:
from transformers import AutoformerConfig, AutoformerModel
import os
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoformerConfig, AutoformerForPrediction
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import timesfm
import pandas as pd

/home/nwertheim/Master-Thesis/timesfm-env/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.7 (main, Oct  9 2024, 00:00:00) [GCC 11.4.1 20231218 (Red Hat 11.4.1-3)].


In [3]:
save_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA.npy")
target_data = np.load(save_dir, allow_pickle=True)
print(len(target_data))
print(target_data[0])
num_none_preterm = sum(1 for item in target_data if item['preterm'] is None)
print(f"Number of instances with None in 'preterm': {num_none_preterm}")
target_data = [item for item in target_data if item['preterm'] is not None]
print(f"Remaining instances after deletion: {len(target_data)}")

target_data = pd.DataFrame(target_data)
print(type(target_data))



1002
{'record_name': 'Hopper-2023_05_25_12_03_38-0000010090-0003-chan0', 'signal': array([[-1.70320951],
       [-3.4187519 ],
       [-2.48399421],
       ...,
       [ 0.73116871],
       [ 0.4095313 ],
       [ 0.95039407]], shape=(65820, 1)), 'fs': 20, 'preterm': 0}
Number of instances with None in 'preterm': 144
Remaining instances after deletion: 858
<class 'pandas.core.frame.DataFrame'>


In [4]:
formatted_data = [
    {
        "start": pd.Timestamp(datetime.now()),  # Replace with actual timestamp if you have it
        "target": row["signal"].astype(float),
        "item_id": row["record_name"]
    }
    for _, row in target_data.iterrows()
]
print(formatted_data[0])

NameError: name 'datetime' is not defined

In [16]:
from sklearn.model_selection import train_test_split

# First, split into 80% train and 20% temp (val + test)
train_data, temp_data = train_test_split(
    formatted_data, test_size=0.2, random_state=42
)

# Then split the 20% temp into 50% val, 50% test => 10% each
val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42
)

print(f"Train: {len(train_data)} samples")
print(f"Validation: {len(val_data)} samples")
print(f"Test: {len(test_data)} samples")


Train: 686 samples
Validation: 86 samples
Test: 86 samples


In [10]:
tfm = timesfm.TimesFm(
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

TypeError: TimesFmBase.__init__() missing 2 required positional arguments: 'hparams' and 'checkpoint'